In [1]:
addprocs(8)
using AuthorGenderStudy

In [2]:
using Embeddings
using MLDataUtils


o_embedding_table = load_embeddings(Word2Vec) 

# Add OOV zero vector
const embedding_table = Embeddings.EmbeddingTable(
    [o_embedding_table.embeddings zeros(300)],
    [o_embedding_table.vocab; "<OOV>"]
)

const enc = LabelEnc.NativeLabels("<OOV>", embedding_table.vocab)
to_ind(lbl, enc=enc) = convertlabel(LabelEnc.Indices, lbl , enc)

to_ind (generic function with 2 methods)

In [ ]:
#const raw_data = collect(Iterators.take(load_data(), 10_000));
const raw_data = collect(load_data());

In [ ]:
train, test = stratifiedobs(first, raw_data, shuffle=true);

In [ ]:
input_rep(datum) = sum(embedding_table.embeddings[:, to_ind(datum.paper_abstract)],2)
input_rep(data::AbstractVector) =  reduce(hcat, input_rep.(data))

In [ ]:
using LIBSVM

In [ ]:
model = svmtrain(input_rep(train), targets(first, train))

In [ ]:
preds, dvs = svmpredict(model, input_rep(test))

In [ ]:
mean(preds .== targets(first, test))

In [ ]:
mean(targets(first, test).==NameToGender.Male)